# Агрегирование и группировка 

## Данные о планетах

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
planets = sns.load_dataset('planets')  # информация об открытых астрономами планетах, вращающихся вокруг других звезд - экзопланет
planets.shape

(1035, 6)

In [2]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


Этот набор данных содержит определенную информацию о более чем 1000 экзопланет, открытых до 2014 года.

## Простое агрегирование в библиотеке Pandas 

Как и в случае одномерных массивов библиотеки NumPy,
для объектов Series библиотеки Pandas агрегирующие функции возвращают скалярное значение:

In [3]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [4]:
ser.sum() # сутта

2.811925491708157

In [6]:
ser.mean() # среднее

0.5623850983416314

В случае объекта DataFrame по умолчанию агрегирующие функции возвращают
сводные показатели по каждому столбцу:

In [7]:
df = pd.DataFrame({'A': rng.rand(5), 'B': rng.rand(5)})
df

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


In [8]:
df.mean()

A    0.477888
B    0.443420
dtype: float64

Можно вместо этого агрегировать и по строкам, задав аргумент axis:

In [9]:
df.mean(axis='columns')

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

Объекты Series и DataFrame библиотеки Pandas содержат методы, соответствующие распространенным агрегирующим функциям. В них есть
удобный метод describe(), вычисляющий сразу несколько самых распространенных сводных показателей для каждого столбца и возвращающий результат.
Опробуем его на наборе данных «Планеты», пока удалив строки с отсутствующими
значениями:

In [13]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


Эта возможность очень удобна для первоначального знакомства с общими характеристиками нашего набора данных. Например, мы видим в столбце year, что,
хотя первая экзопланета была открыта еще в 1989 году, половина всех известных
экзопланет открыта не ранее 2010 года. В значительной степени мы обязаны этим
миссии «Кеплер», представляющей собой космический телескоп, специально разработанный для поиска затмений от планет, вращающих вокруг других звезд.

Для более глубокого исследования данных простых сводных показателей часто
недостаточно. Следующий уровень обобщения данных — операция groupby, позволяющая быстро и эффективно вычислять сводные показатели по подмножествам
данных.

## GroupBy: разбиение, применение, объединение

Простые агрегирующие функции дают возможность «прочувствовать» набор данных, но зачастую бывает нужно выполнить условное агрегирование по какой-либо
метке или индексу. Это действие реализовано в так называемой операции GroupBy.
Название group by («сгруппировать по») ведет начало от одноименной команды
в языке SQL баз данных, но, возможно, будет понятнее говорить о ней в терминах,
придуманных Хэдли Викхэмом, более известным своими библиотеками для языка R: разбиение, применение и объединение.

In [15]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'], 'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


Простейшую операцию «разбить, применить, объединить» можно реализовать
с помощью метода groupby() объекта DataFrame, передав в него имя желаемого
ключевого столбца:

In [16]:
df.groupby('key')

Возвращаемое — не набор объектов DataFrame, а объект
DataFrameGroupBy. Этот объект особенный, его можно рассматривать как специальное представление объекта DataFrame, готовое к группировке, но не выполняющее
никаких фактических вычислений до этапа применения агрегирования. Подобный
метод «отложенного вычисления» означает возможность очень эффективной
реализации распространенных агрегирующих функций, причем практически прозрачным для пользователя образом.
Для получения результата можно вызвать один из агрегирующих методов этого
объекта DataFrameGroupBy, что приведет к выполнению соответствующих шагов
применения/объединения:

In [17]:
df.groupby('key').sum() # сумма

,data
key,
A,3
B,5
C,7


In [21]:
df.groupby('key').prod() # умножение

,data
key,
A,0
B,4
C,10


In [22]:
df.groupby('key').mean()  # среднее

,data
key,
A,1.5
B,2.5
C,3.5


Здесь можно
использовать практически любую распространенную агрегирующую функцию
библиотек Pandas или NumPy, равно как и практически любую корректную операцию объекта DataFrame.

### Объект GroupBy 

#### Индексация по столбцам 

In [26]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [23]:
planets.groupby('method')

In [24]:
planets.groupby('method')['orbital_period']

Здесь мы выбрали конкретную группу Series из исходной группы DataFrame, сославшись на соответствующее имя столбца. Как и в случае с объектом GroupBy,
никаких вычислений не происходит до вызова для этого объекта какого-нибудь
агрегирующего метода:

In [28]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

Результат дает нам общее представление о масштабе чувствительности каждого из
методов к периодам обращения (в днях).

#### Цикл по группам 

Объект GroupBy поддерживает непосредственное выполнение циклов по группам с возвратом каждой группы в виде объекта Series или DataFrame:

In [36]:
for (method, group) in planets.groupby('method'):
    print('{0:30s} shape={1}'.format(method, group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


Это может пригодиться для выполнения некоторых вещей вручную, хотя обычно
быстрее воспользоваться встроенной функциональностью apply.

#### Методы диспечеризации

Благодаря определенной магии классов языка Python все
методы, не реализованные явным образом объектом GroupBy, будут передаваться далее
и выполняться для групп, вне зависимости от того, являются ли они объектами Series
или DataFrame. Например, можно использовать метод describe() объекта DataFrame
для вычисления набора сводных показателей, описывающих каждую группу в данных:

In [48]:
pd.set_option('display.max_rows', None) # показать все строки в выводе
planets.groupby('method')['year'].describe().unstack()

       method                       
count  Astrometry                          2.000000
       Eclipse Timing Variations           9.000000
       Imaging                            38.000000
       Microlensing                       23.000000
       Orbital Brightness Modulation       3.000000
       Pulsar Timing                       5.000000
       Pulsation Timing Variations         1.000000
       Radial Velocity                   553.000000
       Transit                           397.000000
       Transit Timing Variations           4.000000
mean   Astrometry                       2011.500000
       Eclipse Timing Variations        2010.000000
       Imaging                          2009.131579
       Microlensing                     2009.782609
       Orbital Brightness Modulation    2011.666667
       Pulsar Timing                    1998.400000
       Pulsation Timing Variations      2007.000000
       Radial Velocity                  2007.518987
       Transit             

Эта таблица позволяет получить лучшее представление о наших данных. Например, большинство планет было открыто методом измерения лучевой скорости
(radial velocity method) и транзитным методом (transit method), хотя последний
стал распространенным благодаря новым более точным телескопам только в последнее десятилетие. Похоже, что новейшими методами являются метод вариации
времени транзитов (transit timing variation method) и метод модуляции орбитальной яркости (orbital brightness modulation method), которые до 2011 года не использовались для открытия новых планет.

Это всего лишь один пример полезности методов диспетчеризации. Обратите
внимание, что они применяются к каждой отдельной группе, после чего результаты
объединяются в объект GroupBy и возвращаются. Можно использовать для соответствующего объекта GroupBy любой допустимый метод объектов Series/DataFrame,
что позволяет выполнять многие весьма гибкие и мощные операции!

### Агрегирование, фильтрация, преобразование, применение

Предыдущее обсуждение касалось агрегирования применительно к операции объединения, но доступны и другие возможности. В частности, у объектов GroupBy
имеются методы aggregate(), filter(), transform() и apply(), эффективно выполняющие множество полезных операций до объединения сгруппированных данных.

In [49]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                  columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


#### Агрегирование 

Метод aggregate() может принимать на входе строку, функцию
или список и вычислять все сводные показатели сразу. Вот пример, включающий
все вышеупомянутое:

In [54]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

Еще один удобный паттерн — передача в него словаря, связывающего имена столбцов с операциями, которые должны быть применены к этим столбцам:

In [55]:
df.groupby('key').aggregate({'data1': 'min', 'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9
